<img src="imgs/mccoy.png" align="left" width=100/>

---
--- 
# MCCOY
## Millimiter Characterization of Complex Organics in Young stellar objects
--- 
---
#### Written and updated by Carlos E. Muñoz-Romero (2020)


#### Import dependencies

In [2]:
import dynesty
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig

from dynesty import plotting as dyplot
from dynesty import utils as dyfunc
from numpy import random

from lmfit import Model, Parameters
from scipy.stats import norm, truncnorm
%matplotlib qt5

#### Function definitions

In [3]:
'''APPLY REDSHIFT TO SPECTRA'''
def redshift(spectrum, velocity):
    z = np.sqrt( (1 + (velocity/c)) / (1 - (velocity/c)) ) - 1
    return spectrum/(1+z)

'''CONVERT FREQUENCY TO VELOCITY IN KM/S'''
def freq_to_vel(spectrum, reference_frequency):
    return (np.array((c*reference_frequency - c*spectrum) / reference_frequency) )/1000

'''IDENTIFY EMISSION LINES'''
def identify(frequencies, velocities, intensities, catalog):
    
    peaks = sig.find_peaks(intensities)[0]
    peak_intensities = intensities[peaks]
    peak_frequencies = frequencies[peaks]
    
    # TAKE ONLY PEAKS ABOVE 5 SIGMA
    peak_frequencies = peak_frequencies[peak_intensities>=5*rms]
    peak_intensities = peak_intensities[peak_intensities>=5*rms]
    peak_velocities = freq_to_vel(peak_frequencies, reference_frequency)

    detection = dict({"frequencies":[], 
                      "velocities":[], 
                      "catalog_frequencies":[],
                      "eup":[],
                      "Smu2":[],
                      "gu":[],
                      "logaij":[],
                      "transition":[]})
    
    for i,peak in enumerate(peak_frequencies):
        for j,line in enumerate(catalog["frequencies"]):
            if abs(peak-line)<=0.2 and "F" not in catalog["transition"][j]:
                detection["frequencies"].append(peak_frequencies[i])
                detection["velocities"].append(peak_velocities[i])
                detection["catalog_frequencies"].append(catalog["frequencies"][j])
                detection["eup"].append(catalog["eup"][j])
                detection["gu"].append(catalog["gu"][j])
                detection["logaij"].append(catalog["logaij"][j])
                detection["Smu2"].append(catalog["Smu2"][j])
                detection["transition"].append(catalog["transition"][j])
                
    return detection

'GENERATE SPECTRAL WINDOWS AROUND EACH DETECTED TRANSITION'
def generate_windows(velocities, intensities, detections):
    windows = []

    for det_velocity in detections["velocities"]:
        window = dict({"velocities":[],
                       "intensities":[]})
        v = velocities[abs(velocities-det_velocity)<window_size]
        i = intensities[abs(velocities-det_velocity)<window_size]
        window["velocities"] = v-det_velocity
        window["intensities"] = i
        windows.append(window)
        
    return windows

'IDENTIFY POINTS WHICH MAKE UP THE EMISSION LINE'

# Pick all monotonically decreasing neighbors of each detected transition as a line to fit.
def idlines(windows, detections):
    lines = []
    for i,window in enumerate(windows):
        neighbors = dict({"velocities":[],
                          "intensities":[],
                          "indices":[],
                          "line_idx":0})
        line_idx = np.where(window["velocities"]+detections["velocities"][i] == detections["velocities"][i])[0][0]
        neighbors['line_idx'] = line_idx
        # Left neighbors
        diff = 0
        i = 1
        current = line_idx
        while diff <= 0 and abs(window["velocities"][current-i])<line_FWHM*5:
            current = line_idx-i
            neighbors["velocities"].append(window["velocities"][current])
            neighbors["intensities"].append(window["intensities"][current])
            neighbors["indices"].append(current)
            diff = window["intensities"][current-1]-window["intensities"][current]
            i+=1
        # Right neighbors
        diff = 0
        i = 0
        current = line_idx
        while diff <= 0 and abs(window["velocities"][current+i])<line_FWHM*5:
            current = line_idx+i
            neighbors["velocities"].append(window["velocities"][current])
            neighbors["intensities"].append(window["intensities"][current])
            neighbors["indices"].append(current)
            diff = window["intensities"][current+1]-window["intensities"][current]
            i+=1
        lines.append(neighbors)
    return lines

def gaussian(x, amp, center, fwhm):
    sigma = fwhm/(2*np.sqrt(2*np.log(2)))
    return amp / (np.sqrt(2*np.pi) * sigma) * np.exp(-(x-center)**2 / (2*(sigma**2)))

def polynomial_1d(x, m, b):
    return m*x + b

def emission_line_fitter(x, amp, center, fwhm, m, b):
    return gaussian(x, amp, center, fwhm)*is_line + polynomial_1d(x, m, b)*use_baseline

def emission_line(x, amp, center, fwhm, m, b):
    return gaussian(x, amp, center, fwhm) + polynomial_1d(x, m, b)

def emission_lmfit(x, y, is_line):
    
    emission_y = y*is_line
    gmodel = Model(emission_line_fitter)
    params = Parameters()
    params.add_many(('amp', 0.025, True, 0, 1, None, None),
                    ('center', 0, False, -1, 1, None, None),
                    ('fwhm', 0.5, True, 0, 3, None, None),
                    ('m', 0, True, -1, 1, None, None),
                    ('b', 0, True, -0.02, 0.1, None, None))
    
    result = gmodel.fit(y, params, x=x)
    return result

def get_tau(nugu, detections, T):
    c_cgs = c*100
    widths_cgs = widths*100000
    nupper = nugu*np.array(detections["gu"])
    aij = 10**np.array(detections["logaij"])
    freqs_cgs = np.array(detections["catalog_frequencies"])*1e6
    
    tau = np.sqrt(4*np.log(2)/np.pi) * (nupper*aij*c_cgs**3)/(widths_cgs*8*np.pi*freqs_cgs**3)
    tau = tau * (np.exp(h*freqs_cgs / (k*T)) - 1)
    return tau 

def rotdiag(x, log10Ntot, T):
    
    tau = get_tau(nugu, detections, T)
    Ntot = 10**log10Ntot
    C = tau / (1 - np.exp(-tau))
    partitionf = Q(T)
   
    if partitionf < 1:
        partitionf = 1
    C = np.array([c if c>0 else 0 for c in C])
    return np.log(Ntot) - np.log(partitionf) - np.log(C) - (1/T)*x

def rotdiag_lmfit(x, y):
    
    rmodel = Model(rotdiag)
    params = Parameters()
    params.add_many(('log10Ntot', 14, True, 8, 16, None, None),
                    ('T', 20, True, 1, 150, None, None))
    
    result = rmodel.fit(y, params, x=x)
    return result


#### User input
The pipeline is compatible with tab-separated spectroscopy files generated by the NRAO splatalogue database, with frequency in MHz. (https://www.cv.nrao.edu/php/splat/). It is necessary to add extra columns to this file to estimate the rotational partition function via interpolation, if desired.

In [6]:
'SPLATALOGUE FILE'
molecule_name = "CH3CN"
catalog_file = "./splatalogue/CH3CN_catalog.tsv"
spectrum_file = "./181_WSW_FTS200_3mm_average_data_Tmb.dat"
reference_frequency = 93750.7299 
'ESTIMATE OF FULL WIDTH AT HALF MAXIMUM OF EMISSION LINES IN [km/s]'
line_FWHM = 0.5
'VELOCITY OF THE SOURCE IN [km/s]'
v_lsr = 0
'ROOT MEAN SQUARE OF INTENSITY DATA IN [K]'
rms = 5e-3
'SIZE OF SPECTRAL WINDOW TO USE AROUND EACH DETECTED TRANSITION IN [km/s]'
window_size = 25
'CONSTANTS'
c =  2.998*1e8 # m/s
k = 1.3807 * 1e-16 # erg/K 
h = 6.62607 * 1e-27 # erg s
toHz = 1e6

#### Load data and transform to velocity units (km/s)

In [7]:
spectrum = np.loadtxt(spectrum_file)
frequencies = redshift(spectrum[:,0], v_lsr)
velocities = freq_to_vel(frequencies, reference_frequency)
intensities = spectrum[:,1]
catalog_dataframe = pd.read_csv(catalog_file, delimiter="\t", header=0, index_col=False)
catalog = dict({"frequencies":catalog_dataframe["Freq-MHz(rest frame,redshifted)"], 
                "velocities":freq_to_vel(catalog_dataframe["Freq-MHz(rest frame,redshifted)"], reference_frequency),
                "Smu2":catalog_dataframe["S<sub>ij</sub>&#956;<sup>2</sup> (D<sup>2</sup>)"],
                "eup":catalog_dataframe["E_U (K)"],
                "gu":catalog_dataframe["Upper State Degeneracy"],
                "logaij":catalog_dataframe["Log<sub>10</sub> (A<sub>ij</sub>)"],
                "transition":catalog_dataframe["Resolved QNs"],
                "Qrot":catalog_dataframe["Q"],
                "Trot":catalog_dataframe["T"],
                "linelist":catalog_dataframe["Linelist"]})

## Line Fitting

#### Find all emission lines

In [9]:
detections = identify(frequencies, velocities, intensities, catalog)
detections

{'frequencies': [73588.7747,
  73590.3372,
  91985.2624,
  91987.0202,
  110381.359,
  110383.508],
 'velocities': [64474.7424943515,
  64469.74586658658,
  5645.685714282595,
  5640.064548020122,
  -53182.12038987016,
  -53188.99255183292],
 'catalog_frequencies': [73588.7994,
  73590.2183,
  91985.3141,
  91987.0876,
  110381.372,
  110383.4999],
 'eup': [15.97536, 8.82955, 20.39005, 13.24426, 25.68755, 18.54192],
 'Smu2': [79.48062, 84.80078, 101.7415, 105.98313, 123.64362, 127.17758],
 'gu': [18.0, 18.0, 22.0, 22.0, 26.0, 26.0],
 'logaij': [-4.68864,
  -4.660480000000001,
  -4.37783,
  -4.3600699999999994,
  -4.12818,
  -4.11592],
 'transition': ['4(1)-3(1)',
  '4(0)-3(0)',
  '5(1)-4(1)',
  '5(0)-4(0)',
  '6(1)-5(1)',
  '6(0)-5(0)']}

#### Define spectral windows and center around zero

In [10]:
windows = generate_windows(velocities, intensities, detections)
lines = idlines(windows, detections)

In [11]:
for window in windows:
    plt.figure()
    plt.step(window["velocities"], window["intensities"])

#### Perform initial fit of emission lines via non-linear least-squares minimization. Then, estimate uncertainties via nested sampling of the posterior distribution, using truncated normal priors centered around the lmfit results

In [12]:
def loglike(theta):
    amp, fwhm, lnf = theta
    m = lmfit_m
    b = lmfit_b
    center = 0 
   
    model = emission_line(x, amp, center, fwhm, m, b)
    inv_sigma2 = 1.0 / (yerr**2 + model**2 * np.exp(2 * lnf))
    return -0.5 * (np.sum((y-model)**2 * inv_sigma2 - np.log(inv_sigma2)))

line_results_dynesty = []
line_results_lmfit = []

for i in range(len(lines)):
    is_line = np.zeros(len(windows[i]["velocities"]))
    is_line[lines[i]["indices"]] = 1

    x = windows[i]["velocities"]
    y = windows[i]["intensities"]
    
    use_baseline = (y<3*rms).astype(int)
    
    lmfit_result = emission_lmfit(x, y, is_line)
    line_results_lmfit.append(lmfit_result)
    
    lmfit_amp = lmfit_result.best_values["amp"]
    lmfit_center = lmfit_result.best_values["center"]
    lmfit_fwhm = lmfit_result.best_values["fwhm"]
    lmfit_m = lmfit_result.best_values["m"]
    lmfit_b = lmfit_result.best_values["b"]
    
    x = x[is_line==1]
    y = y[is_line==1]
    y = [k if k>0 else 0 for k in y]
    yerr = random.randn(len(y))*0.001
    
    def prior_transform(utheta):
        uamp, ufwhm, ulnf = utheta
        # Truncated Normal
        mean_amp , s_amp  = lmfit_amp, 0.2*lmfit_amp  # mean and standard deviation
        low_amp , high_amp  = 0, 2*lmfit_amp  
        low_n_amp, high_n_amp = (low_amp - mean_amp ) / s_amp , (high_amp - s_amp ) / s_amp  # standardize
        amp = truncnorm.ppf(uamp, low_n_amp, high_n_amp, loc=mean_amp, scale=s_amp)
        # Truncated Normal
        mean_fwhm , s_fwhm  = lmfit_fwhm, 0.2*lmfit_fwhm  # mean and standard deviation
        low_fwhm , high_fwhm  = 0, 3
        low_n_fwhm, high_n_fwhm = (low_fwhm - mean_fwhm ) / s_fwhm , (high_fwhm - s_fwhm ) / s_fwhm  # standardize
        fwhm = truncnorm.ppf(ufwhm, low_n_fwhm, high_n_fwhm, loc=mean_fwhm, scale=s_fwhm)
        
        lnf = 15. * ulnf - 10.
        return amp, fwhm, lnf
    
    dsampler = dynesty.DynamicNestedSampler(loglike, prior_transform, ndim=3,
                                            bound='multi', sample='auto')
    dsampler.run_nested(dlogz_init=0.01)
    dres = dsampler.results
    line_results_dynesty.append(dres)

19724it [04:07, 79.59it/s, batch: 10 | bound: 33 | nc: 13 | ncall: 173538 | eff(%): 11.366 | loglstar: 15.843 < 18.430 < 17.432 | logz: 14.120 +/-  0.077 | stop:  0.916]       
17497it [03:39, 79.57it/s, batch: 8 | bound: 34 | nc: 12 | ncall: 173328 | eff(%): 10.095 | loglstar: 15.181 < 24.526 < 23.720 | logz: 19.665 +/-  0.110 | stop:  0.968]           
24999it [06:00, 69.37it/s, batch: 13 | bound: 37 | nc: 1 | ncall: 231855 | eff(%): 10.782 | loglstar:   -inf < 23.508 < 22.911 | logz: 19.585 +/-  0.077 | stop:  0.796]        
20181it [04:34, 73.51it/s, batch: 11 | bound: 29 | nc: 1 | ncall: 180447 | eff(%): 11.184 | loglstar: 19.576 < 21.991 < 21.476 | logz: 18.144 +/-  0.077 | stop:  0.959]         
21344it [04:42, 75.57it/s, batch: 11 | bound: 36 | nc: 2 | ncall: 193162 | eff(%): 11.050 | loglstar: 23.506 < 26.201 < 25.514 | logz: 21.635 +/-  0.078 | stop:  0.787]        
20884it [04:26, 78.40it/s, batch: 10 | bound: 33 | nc: 2 | ncall: 178227 | eff(%): 11.718 | loglstar: 23.356 < 

In [13]:
fluxes = []    
widths = []    

flux_errors = []
widths_errors = []

for i,dres in enumerate(line_results_dynesty):
    
    x = windows[i]["velocities"]
    y = windows[i]["intensities"]
    
    is_line = np.zeros(len(windows[i]["velocities"]))
    is_line[lines[i]["indices"]] = 1
    
    # Extract sampling results.
    samples = dres.samples  # samples
    weights = np.exp(dres.logwt - dres.logz[-1])  # normalized weights
    
    lmfit_result = line_results_lmfit[i]
    lmfit_amp = lmfit_result.best_values["amp"]
    lmfit_center = lmfit_result.best_values["center"]
    lmfit_fwhm = lmfit_result.best_values["fwhm"]
    lmfit_m = lmfit_result.best_values["m"]
    lmfit_b = lmfit_result.best_values["b"]
    
    
    # Compute quantiles.
    quantiles = [dyfunc.quantile(samps, [0.1586, 0.5, 0.84135], weights=weights)
                 for samps in samples.T]
    
    dynesty_flux = quantiles[0][1]
    dynesty_fwhm = quantiles[1][1]
    
    fluxes.append(dynesty_flux)
    widths.append(dynesty_fwhm)
    flux_errors.append([dynesty_flux-quantiles[0][0], quantiles[0][2]-dynesty_flux])
    widths_errors.append([dynesty_fwhm-quantiles[1][0], quantiles[1][2]-dynesty_fwhm])
    
    plt.figure()
    window_linspace = np.linspace(min(windows[i]["velocities"]), max(windows[i]["velocities"]),10000)
    plt.step(x, y, color="gray")
    plt.scatter(x,y*is_line)
    plt.plot(window_linspace, emission_line(window_linspace, lmfit_amp,
                                        lmfit_center, lmfit_fwhm, lmfit_m, lmfit_b),color="red")
    plt.plot(window_linspace, emission_line(window_linspace, dynesty_flux,
                                        lmfit_center, dynesty_fwhm, lmfit_m, lmfit_b),color="green")



In [14]:
labels = [r'$\int T dv$ ', r'FWHM ',r'$\ln f$']
for dres in line_results_dynesty:    
    #fig, axes = dyplot.traceplot(dres, labels=labels, title_fmt=(".4f"),
                                 #trace_cmap='inferno', post_color="black", label_kwargs=dict({"fontsize":15}))
    fig, axes = dyplot.cornerplot(dres, color='dodgerblue', show_titles=True, title_fmt=(".4f"), labels=labels)


## Rotation Diagram

In [15]:
strengths = np.array(detections["Smu2"]) * (1e-18)**2 # from D^2 to (statC cm)^2
fluxes = np.array(fluxes)
fluxes_cgs = fluxes*100000 # in cm/s
widths = np.array(widths)
flux_errors = np.array(flux_errors).reshape(len(lines),2)
flux_errors_cgs = np.array(flux_errors).reshape(len(lines),2)*100000 
widths_errors = np.array(widths_errors).reshape(len(lines),2)
# Upper-level populations Nu/gu
nugu = (3*k*fluxes_cgs)/(8*(np.pi**3)*np.array(detections["catalog_frequencies"])*toHz*strengths)
lnnugu = np.log(nugu)
lnnugu_err = flux_errors / np.array([[f,f] for f in fluxes]).reshape(len(lines),2)


In [16]:
Q_array = catalog["Qrot"]
T_array  = catalog["Trot"]
Q_array = Q_array[Q_array == Q_array]
T_array = T_array[T_array == T_array]

'FIT A POLYNOMIAL TO THE PARTITION FUNCTION DATA TO INTERPOLATE'
Qfit = np.polyfit(T_array, Q_array, 3)
Q = np.poly1d(Qfit)

x = np.array(detections["eup"])
y = lnnugu
yerr = lnnugu_err.T

lmfit_result = rotdiag_lmfit(x, y)    
lmfit_T = lmfit_result.best_values["T"]
lmfit_log10Ntot = lmfit_result.best_values["log10Ntot"]
lmfit_result

In [21]:
# log-likelihood
def loglike(theta):
    
    T, log10Ntot, lnf = theta
    
    model = rotdiag(x,log10Ntot,T)
    inv_sigma2 = 1.0 / (yerr**2 + model**2 * np.exp(2 * lnf))
    return -0.5 * (np.sum((y-model)**2 * inv_sigma2 - np.log(inv_sigma2)))

# prior transform
def prior_transform(utheta):
    uT, ulog10Ntot, ulnf = utheta
    
    # Truncated Normal
    mean_T , s_T  = lmfit_T, 0.5*lmfit_T  # mean and standard deviation
    low_T , high_T  = 1, 2*lmfit_T
    low_n_T, high_n_T = (low_T - mean_T ) / s_T , (high_T - s_T ) / s_T  # standardize
    T = truncnorm.ppf(uT, low_n_T, high_n_T, loc=mean_T, scale=s_T)
    # Truncated Normal
    mean_log10Ntot , s_log10Ntot  = lmfit_log10Ntot, 0.5*lmfit_log10Ntot  # mean and standard deviation
    low_log10Ntot , high_log10Ntot  = 8, 1.5*lmfit_log10Ntot
    low_n_log10Ntot, high_n_log10Ntot = (low_log10Ntot - mean_log10Ntot ) / s_log10Ntot , (high_log10Ntot - s_log10Ntot ) / s_log10Ntot  # standardize
    log10Ntot = truncnorm.ppf(ulog10Ntot, low_n_log10Ntot, high_n_log10Ntot, loc=mean_log10Ntot, scale=s_log10Ntot)
   
    #T = uT*148 + 2
    #log10Ntot = ulog10Ntot*8+8
    lnf = 21 * ulnf - 20.
    return T, log10Ntot, lnf

dsampler = dynesty.DynamicNestedSampler(loglike, prior_transform, ndim=3, bound='multi', sample='auto')
dsampler.run_nested(dlogz_init=0.01)
rotdiag_res = dsampler.results

21564it [03:42, 96.95it/s, batch: 10 | bound: 73 | nc: 6 | ncall: 79426 | eff(%): 27.150 | loglstar:  0.462 <  7.793 <  7.293 | logz: -0.679 +/-  0.155 | stop:  0.944]               


In [22]:
# Extract sampling results.
samples = rotdiag_res.samples  # samples
weights = np.exp(rotdiag_res.logwt - rotdiag_res.logz[-1])  # normalized weights
quantiles = [dyfunc.quantile(samps, [0.025, 0.1586, 0.5, 0.84135, 0.975], weights=weights)
                 for samps in samples.T]

labels = [r'T$_{rot}$ ', r'log$_{10}$(N$_{tot}$)']
fig, axes = dyplot.cornerplot(rotdiag_res, color='brown', show_titles=True, dims=[0,1],
                              title_fmt=(".4f"), labels=labels, label_kwargs=dict({"fontsize":15}))

In [23]:
plt.errorbar(x,y,yerr=lnnugu_err.T, fmt="o",color="black",capsize=4)
alphas = weights/max(weights)*0.1
eulinspace = np.linspace(0,max(x)+10)
sorted_x = np.array(sorted(list(x)))

for i,sample in enumerate(rotdiag_res.samples[::10]):
    plt.plot(sorted_x, rotdiag(sorted_x, sample[1], sample[0]),alpha=0.1,color="dodgerblue", linewidth=0.05)
plt.plot(sorted_x, rotdiag(sorted_x, quantiles[1][2], quantiles[0][2]),
         color="black",linestyle="--",linewidth=2)
plt.show()


In [28]:
np.save("./fits/{}_lines_dynesty".format(molecule_name), line_results_dynesty)
#np.save("./fits/{}_lines_lmfit".format(molecule_name), line_results_lmfit)
np.save("./fits/{}_rotdiag_lmfit".format(molecule_name), lmfit_result)
np.save("./fits/{}_rotdiag_dynesty".format(molecule_name), dres)
np.save("./fits/{}_detection_info".format(molecule_name), detections)

In [27]:
get_tau(nugu, detections, quantiles[0][2])

array([0.00279424, 0.0041505 , 0.00413099, 0.00496125, 0.00200937,
       0.00233058])